# 데이터 merge code 

- 작성자: 이육샛별, 박하람
- 참고자료: [PinkWink의 파이썬으로 데이터 주무르기](https://github.com/PinkWink/DataScience)
- jupyter notebook으로 작성

In [ ]:
import pandas as pd
import numpy as np
import re 
from glob import glob

In [ ]:
def cut_char_sigu(name): 
    return name if len(name)==2 else name[:-1]

In [ ]:
def make_ID_column(df): 
    sido_candi = df['지역']
    sido_candi = [name[:2] if name[:2] in ['서울','부산','대구','광주','인천','대전','울산']
             else '' for name in sido_candi]
    
    sigun_candi = ['']*len(df) #row수만큼 칸 만들기
    
    for n in df.index: 
        each = df['행정구역'][n]
        if each[:2] in ['창원']:
            sigun_candi[n] = re.split('시', each)[0]+' '+ cut_char_sigu(re.split('시',each)[1])
        else: 
            sigun_candi[n] = cut_char_sigu(each)
    
    # 광역시도 이름과 시구군 이름을 합침 
    ID_candi = [sido_candi[n]+' '+sigun_candi[n] for n in range(0, len(sigun_candi))]
    # 광역시가 아니면 ' '가 처음에 나오니 ' ' 다음 글자가 나오게 함.
    ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
    # 세종시의 경우는 예외로 처리해 '세종'만 나오게! 
    ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi]

    df['ID'] = ID_candi
    
    df.loc[122, 'ID'] = '고성(강원)'
    df.loc[224, 'ID'] = '고성(경남)'
    df.loc[218, 'ID'] = '창원 합포'
    df.loc[219, 'ID'] = '창원 회원'
    
    return df

In [ ]:
def big_ind_cate(df1, df2, df3): 
    patients = make_ID_column(df1)
    pop = make_ID_column(df2)
    ind = make_ID_column(df3)
    
    df_merge = pd.merge(pop, patients)
    df = pd.merge(df_merge, ind)
    
    # 필요한 칼럼만 정리 
    df = df[['산업분류','ID','지역','행정구역','인구 수','확진자 수','총사업체수','총종사자수','상용 종사자수','임시및일용 종사자수']]
    
    # 필요한 칼럼 생성 
    df['인구 대비 확진자 수'] = df['확진자 수'] / df['인구 수']
    df['인구 대비 산업체 수'] = df['총사업체수'] / df['인구 수']
    df['인구 대비 산업체 종사자 수'] = df['총종사자수'] / df['인구 수']
    df['종사자수 대비 산업체 수'] = df['총사업체수'] / df['총종사자수']
    df['종사자수 대비 상용 종사자 수'] = df['상용 종사자수'] / df['총종사자수']
    df['종사자수 대비 임시및일용 종사자 수'] = df['임시및일용 종사자수'] / df['총종사자수']
    df['종사자수 대비 비정규 종사자 비'] = df['종사자수 대비 임시및일용 종사자 수'] / df['종사자수 대비 상용 종사자 수']
    df[['인구 대비 확진자 수','인구 대비 산업체 수','인구 대비 산업체 종사자 수','종사자수 대비 산업체 수',
       '종사자수 대비 상용 종사자 수','종사자수 대비 임시및일용 종사자 수', '종사자수 대비 비정규 종사자 비']] *= 100
    
    # nan, inf, -inf 처리 
    for column in ['종사자수 대비 산업체 수','종사자수 대비 상용 종사자 수','종사자수 대비 임시및일용 종사자 수', '종사자수 대비 비정규 종사자 비']:
        try: 
            inf_ind_list = list(df[df.isin([np.nan, np.inf, -np.inf]).any(1)][column].index)
        except:
            inf_ind_list = df[df.isin([np.nan, np.inf, -np.inf]).any(1)][column].index
        
        df.loc[inf_ind_list, column] = 0
    
    return df

## 필요한 데이터만 담은 파일 형성

- 시군구별 확진자수 데이터
- 시군구별 인구 데이터
- 대분류 산업별 데이터 

In [ ]:
# 시군구별 확진자수 데이터
patients = pd.read_excel('시군구별-확진자수-해외유입-포함.xlsx', encoding='utf-8')

In [ ]:
# 시군구별 인구수 데이터 
pop = pd.read_excel('인구-시군구-수정.xls', encoding='utf-8')
pop.columns = ['지역','행정구역','인구 수']

In [ ]:
# 시군구별 대분류 산업체 데이터
industries_files = ['지역별 데이터 2_수정.xlsx', '지역별 데이터 7_수정.xlsx',
                   '지역별 데이터 8_수정.xlsx', '지역별 데이터 10_수정.xlsx',
                   '지역별 데이터 14_수정.xlsx', '지역별 데이터 15_수정.xlsx', 
                   '지역별 데이터 16_수정.xlsx', '지역별 데이터 17_수정.xlsx']

In [ ]:
for n, ind in enumerate(industries_files): 
    ind_df = pd.read_excel(f'대분류-산업별-데이터/{ind}', encoding='utf-8')
    ind_df.columns = ['지역','행정구역','코드','산업분류','총사업체수','총종사자수','자영업자 종사자수',
               '무급가족 종사자수','상용 종사자수','임시및일용 종사자수','기타 종사자수']
    
    final_ind_df = big_ind_cate(patients, pop, ind_df)
    
    final_ind_df.to_csv(f'최종-대분류-산업별-데이터-{n}.csv', sep=',', encoding='utf-8')

In [ ]:
# 시군구별 소분류 산업체 데이터 
for n in range(9): 
    ind_df = pd.read_excel(f'소분류-산업별-데이터/세부 산업체별 데이터 {n}_수정.xlsx', encoding='utf-8')
    ind_df.columns = ['지역','행정구역','산업분류','총사업체수','총종사자수','자영업자 종사자수',
               '무급가족 종사자수','상용 종사자수','임시및일용 종사자수','기타 종사자수']
    
    final_ind_df = big_ind_cate(patients, pop, ind_df)
    
    final_ind_df.to_csv(f'최종-소분류-산업별-데이터-{n}.csv', sep=',', encoding='utf-8')